In [1]:
from pyspark import SparkContext
from pyspark.sql.functions import countDistinct
sc=SparkContext()

22/12/21 18:01:05 WARN Utils: Your hostname, Tobiass-MacBook.local resolves to a loopback address: 127.0.0.1; using 192.168.178.88 instead (on interface en0)
22/12/21 18:01:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/21 18:01:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import numpy as np


Pseude Code:

agrupar por user, device, gt
    para x, y, z sacar
        median
        desv
        max
        min

join data from different datasets into RDDs
Analize RDDs

Don´t repeat code!
avoid many (unnecessary) lines of code / repetitions

In [3]:
#columns = ["index", "arrival_time", "creation_time", "x", "y", "z", "user", "model", "device", "activity"]

pa = sc.textFile('Phones_accelerometer.csv')

pg = sc.textFile('Phones_gyroscope.csv')

wa = sc.textFile('Watch_accelerometer.csv')

wg = sc.textFile('Watch_gyroscope.csv')


In [4]:
split = wg.map(lambda x : x.split(","))
keyed = split.map(lambda x : (x[6] + "," + x[7]+ "," + x[9], np.array([1, float(x[3]), float(x[4]), float(x[5]), float(x[3])**2, float(x[4])**2, float(x[5])**2])))

sum = keyed.reduceByKey(lambda x, y : x + y)
std = sum.map(lambda x : (x[0], np.array([np.sqrt((x[1][4]/x[1][0])-(x[1][1]/x[1][0])**2), np.sqrt((x[1][5]/x[1][0])-(x[1][2]/x[1][0])**2), np.sqrt((x[1][6]/x[1][0])-(x[1][3]/x[1][0])**2)])))
mean = sum.map(lambda x : (x[0], np.array((x[1][1]/x[1][0], x[1][2]/x[1][0], x[1][3]/x[1][0]))))

#sum_std = keyed.map(lambda x: (x[1][1]  - mean[1][1])**2)
'''std = sum_std.reduce(lambda x,y: x + y)
std = np.sqrt(std/heights.count())'''

test = keyed.map(lambda x: (x[0], x[1][0]))
test = test.reduceByKey(lambda x, y: max(x,y))

test.collect()

[('a,gear,stand', 1.0), ('a,gear,null', 1.0), ('a,gear,sit', 1.0)]

In [5]:
def read_in(filename):
    #read in file and split at ","
    rdd = sc.textFile(filename)
    split_rdd = rdd.map(lambda x : x.split(","))

    #remove unnecessary variables, create primary key (user + model + gt), create floats: counter variable, x, y, z
    keyed = split_rdd.map(lambda x : (x[6] + "," + x[7]+ "," + x[9], np.array([1, float(x[3]), float(x[4]), float(x[5])])))
    return keyed

In [6]:
out = read_in('Phones_accelerometer.csv')
out.collect()

[('a,nexus4,stand', array([ 1.       , -5.958191 ,  0.6880646,  8.135345 ])),
 ('a,nexus4,stand', array([ 1.       , -5.95224  ,  0.6702118,  8.136536 ])),
 ('a,nexus4,stand', array([ 1.       , -5.9950867,  0.6535492,  8.204376 ])),
 ('a,nexus4,stand', array([ 1.       , -5.9427185,  0.6761627,  8.128204 ])),
 ('a,nexus4,stand',
  array([ 1.        , -5.991516  ,  0.64164734,  8.135345  ])),
 ('a,nexus4,stand', array([ 1.       , -5.965332 ,  0.6297455,  8.128204 ])),
 ('a,nexus4,stand', array([ 1.       , -5.991516 ,  0.6356964,  8.16272  ])),
 ('a,nexus4,stand',
  array([ 1.        , -5.915344  ,  0.63093567,  8.105591  ])),
 ('a,nexus4,stand', array([ 1.       , -5.984375 ,  0.6940155,  8.067505 ])),
 ('a,nexus4,stand',
  array([ 1.        , -5.937958  ,  0.71543884,  8.090118  ])),
 ('a,nexus4,stand', array([ 1.       , -5.902252 ,  0.6678314,  8.069885 ])),
 ('a,nexus4,stand',
  array([ 1.        , -5.9498596 ,  0.68092346,  8.119873  ])),
 ('a,nexus4,stand', array([ 1.       , -

In [7]:
def calc_mean(rdd):
    # sum up numericals from input rdd --> x[1]: count per key, sum(x), sum(y), sum(z)
    sum = rdd.reduceByKey(lambda x, y : x + y)
    # divide sums by count
    mean = sum.map(lambda x : (x[0], np.array((x[1][1]/x[1][0], x[1][2]/x[1][0], x[1][3]/x[1][0]))))

    return mean


In [8]:
def calc_std(rdd):
    #remove unnecessary variables, create primary key (user + model + gt), create floats: counter variable, x, y, z, x^2, y^2, z^2 
    work_rdd = rdd.map(lambda x : (x[0], np.array([1, x[1][1], x[1][2], x[1][3], (x[1][1])**2, (x[1][2])**2, (x[1][3])**2])))

    # sum up individual numerical values
    sum = work_rdd.reduceByKey(lambda x, y : x + y)
    # calculate std as follows: sqrt( (sum(x^2)/N) - (sum(x)/N)^2 )
    std = sum.map(lambda x : (x[0], np.array([np.sqrt((x[1][4]/x[1][0])-(x[1][1]/x[1][0])**2), np.sqrt((x[1][5]/x[1][0])-(x[1][2]/x[1][0])**2), np.sqrt((x[1][6]/x[1][0])-(x[1][3]/x[1][0])**2)])))

    return std

In [9]:
# def find_max_old(rdd):
#     rdd = read_in('Watch_gyroscope.csv')
#     rdd = rdd.filter(lambda x : x[0] == 'a,gear,null')
#     first_line = rdd.take(1)
#     count = rdd.count()
#     print(rdd.count())

#     while count > 0:
#         max_x = first_line[0][1][1]
#         maxed_x = rdd.filter(lambda x :  x[1][1] > max_x )
#         count = maxed_x.count()
#         first_line = maxed_x.take(1)
    
#     count = rdd.count()
#     first_line = rdd.take(1)
#     while count > 0:
#         max_y = first_line[0][1][2]
#         maxed_y = rdd.filter(lambda x :  x[1][2] > max_y )
#         count = maxed_y.count()
#         first_line = maxed_y.take(1)
    
#     count = rdd.count()
#     first_line = rdd.take(1)
#     while count > 0:
#         max_z = first_line[0][1][3]
#         maxed_z = rdd.filter(lambda x :  x[1][3] > max_z )
#         count = maxed_z.count()
#         first_line = maxed_z.take(1)

#     return [max_x, max_y, max_z]

In [10]:
def get_keys(rdd):
    
    out = rdd.reduceByKey(lambda x, y: x+y)


    collected = out.collect()
    keys = []
    for i in range(out.count()):
        keys.append(collected[i][0])

    return keys


In [11]:
# def find_max_old(rdd):
#     rdd = read_in('Watch_gyroscope.csv')
#     keys = get_keys(rdd)
#     out = sc.emptyRDD()
#     #print(out.collect(), "break\n")

#     for i in range(len(keys)):
#         key = keys[i]
#         #print(key)
#         max_values = np.empty([3,1])
#         for i in range(3):
#             current = rdd.filter(lambda x : x[0]==key)
#             #print(current.take(1))
#             current_x = current.map(lambda x: (x[0], x[1][i+1]))
#             max_x = current_x.max()
#             max_values[i] = max_x[1]
#             #print(max_x[1])
        
#         entry = (str(key), max_values)
#         entry_rdd = sc.parallelize([key, max_values])
#         #print(entry_rdd.collect())
#         out = out.union(entry_rdd)
#     #out = out.map(lambda x : (x[0], list(x[1:])))
#     #out.collect()
#     return out

In [12]:
# rdd = read_in('Watch_gyroscope.csv')
# keys = get_keys(rdd)
# out = sc.emptyRDD()

    
# x = rdd.map(lambda x: (x[0], x[1][1]))
# y = rdd.map(lambda x: (x[0], x[1][2]))
# z = rdd.map(lambda x: (x[0], x[1][3]))
# x = x.reduceByKey(lambda x, y: max(x,y))
# y = y.reduceByKey(lambda x, y: max(x,y))
# z = z.reduceByKey(lambda x, y: max(x,y))
# rdd = x.join(y).join(z).mapValues(lambda x: x[0] + (x[1], ))
# #rdd = rdd.map(lambda x : (x[0], np.array(x[1][0], x[1][1], x[2])))
# #rdd = rdd.map(lambda x : (x[0], np.array([x[1][0], x[1][1], x[1][2]])))
# #rdd = rdd.map(lambda x : (x[0], np.array(x[1])))

# rdd.collect()


In [13]:
rdd = read_in('Watch_gyroscope.csv')
#keys = get_keys(rdd)
#out = sc.emptyRDD()

    
x = rdd.map(lambda x: (x[0], x[1][1]))
y = rdd.map(lambda x: (x[0], x[1][2]))
z = rdd.map(lambda x: (x[0], x[1][3]))
x = x.reduceByKey(lambda x, y: min(x,y))
y = y.reduceByKey(lambda x, y: min(x,y))
z = z.reduceByKey(lambda x, y: min(x,y))
rdd = x.join(y).join(z).mapValues(lambda x: x[0] + (x[1], ))
#rdd = rdd.map(lambda x : (x[0], np.array(x[1][0], x[1][1], x[2])))
#rdd = rdd.map(lambda x : (x[0], np.array([x[1][0], x[1][1], x[1][2]])))
#rdd = rdd.map(lambda x : (x[0], np.array(x[1])))

rdd.collect()


[('a,gear,stand', (-2.0383835, -0.55287224, -1.2319783)),
 ('a,gear,null', (-0.0314253, -0.08042747, -0.09347696)),
 ('a,gear,sit', (-0.700944, -0.79042625, -0.22849922))]

In [14]:
# rdd = read_in('Watch_gyroscope.csv')
# keys = get_keys(rdd)
# out = sc.emptyRDD()

    
# rdd = rdd.map(lambda x: (x[0], x[1][1]))
# #rdd = rdd.reduceByKey(lambda x, y: max(x,y))
# #rdd.collect()

# other = find_max(rdd)
# other.collect()

In [15]:
# rdd = read_in('Watch_gyroscope.csv')
# keys = get_keys(rdd)
# #out = rdd.filter(lambda x : x[0] == 'a,gear,null')
# out = find_max(rdd)
# print(out.collect())



# # for i in range(len(keys)):
# #     # print(out.collect()[2*i], out.collect()[2*i+1])
# #     out = out.map(lambda x : (x[i]))

# out = out.map(lambda x : (x[0][0]))
# out.collect()

In [16]:
# rdd = read_in('Watch_gyroscope.csv')
# keys = get_keys(rdd)
# out = sc.emptyRDD()
# #print(out.collect(), "break\n")

# for i in range(len(keys)):
#     key = keys[i]
#     #print(key)
#     max_values = np.empty([3,1])
#     for i in range(3):
#         current = rdd.filter(lambda x : x[0]==key)
#         #print(current.take(1))
#         current_x = current.map(lambda x: (x[0], x[1][i+1]))
#         max_x = current_x.max()
#         max_values[i] = max_x[1]
#         #print(max_x[1])
    
#     entry = (str(key), max_values)
#     entry_rdd = sc.parallelize(entry)
#     #print(entry_rdd.collect())
#     out = out.union(entry_rdd)
# #out = out.map(lambda x : (x[0], list(x[1:])))
# out.collect()

In [17]:
# def find_min_old(rdd):
#     rdd = read_in('Watch_gyroscope.csv')
#     keys = get_keys(rdd)
#     out = sc.emptyRDD()
#     #print(out.collect(), "break\n")

#     for i in range(len(keys)):
#         key = keys[i]
#         #print(key)
#         max_values = np.empty([3,1])
#         for i in range(3):
#             current = rdd.filter(lambda x : x[0]==key)
#             #print(current.take(1))
#             current_x = current.map(lambda x: (x[0], x[1][i+1]))
#             max_x = current_x.min()
#             max_values[i] = max_x[1]
#             #print(max_x[1])
        
#         entry = (str(key), max_values)
#         entry_rdd = sc.parallelize(entry)
#         #print(entry_rdd.collect())
#         out = out.union(entry_rdd)
#     #out = out.map(lambda x : (x[0], list(x[1:])))
#     #out.collect()
#     return out

In [18]:
def find_max(rdd):
        
    x = rdd.map(lambda x: (x[0], x[1][1]))
    y = rdd.map(lambda x: (x[0], x[1][2]))
    z = rdd.map(lambda x: (x[0], x[1][3]))
    x_max = x.reduceByKey(lambda x, y: max(x,y))
    y_max = y.reduceByKey(lambda x, y: max(x,y))
    z_max = z.reduceByKey(lambda x, y: max(x,y))
    max_out = x_max.join(y_max).join(z_max).mapValues(lambda x: x[0] + (x[1], ))

    return max_out

In [19]:
def find_min(rdd):
        
    x = rdd.map(lambda x: (x[0], x[1][1]))
    y = rdd.map(lambda x: (x[0], x[1][2]))
    z = rdd.map(lambda x: (x[0], x[1][3]))
    x = x.reduceByKey(lambda x, y: min(x,y))
    y = y.reduceByKey(lambda x, y: min(x,y))
    z = z.reduceByKey(lambda x, y: min(x,y))
    rdd = x.join(y).join(z).mapValues(lambda x: x[0] + (x[1], ))

    return rdd

In [135]:
def process(filename):
    rdd = read_in(filename)
    mean = calc_mean(rdd)
    std = calc_std(rdd)
    max = find_max(rdd)
    min = find_min(rdd)

    out = mean.join(std).join(max).map(lambda x : (x[0], x[1])).join(min)
    #out = out.map(lambda x : (x[0], x[1][0][0], x[1][1]))
    return out

In [137]:
out = process('Watch_gyroscope.csv')
print(out.collect())

[('a,gear,sit', (((array([ 0.02270562, -0.0414225 , -0.0699732 ]), array([0.05703369, 0.05626924, 0.04523762])), (0.39867523, 0.39920786, 0.77604514)), (-0.700944, -0.79042625, -0.22849922))), ('a,gear,stand', (((array([ 0.02214666, -0.03286732, -0.07143673]), array([0.07314945, 0.05570244, 0.0501775 ])), (0.81039995, 0.35446674, 1.1475562)), (-2.0383835, -0.55287224, -1.2319783))), ('a,gear,null', (((array([ 0.0225701 , -0.03660876, -0.07169494]), array([0.01779986, 0.01156134, 0.00803968])), (0.0561927, -0.018109497, -0.055127434)), (-0.0314253, -0.08042747, -0.09347696)))]


In [105]:
def process_further(filename):
    rdd = read_in(filename)
    mean = calc_mean(rdd)
    std = calc_std(rdd)
    max = find_max(rdd)
    min = find_min(rdd)

    out = mean.join(std).join(max).map(lambda x : (x[0], x[1])).join(min)
    out = out.map(lambda x : (x[0], x[1][0][0][0][0], x[1][0][0][0][1], x[1][0][0][0][2], x[1][0][0][1][0], x[1][0][0][1][1], x[1][0][0][1][2], x[1][0][1][0], x[1][0][1][1], x[1][0][1][2], x[1][1][0], x[1][1][1], x[1][1][2]))
    #out = out.map(lambda x : np.split(x[1], ","))
    #out = out.map(lambda x : (x[0], x[1][0][0], x[1][1]))
    return out

In [138]:
out = process_further('Watch_gyroscope.csv')
print(out.collect())

[('a,gear,sit', 0.02270561507082863, -0.04142249651518432, -0.06997320145904558, 0.057033688206817146, 0.056269236352008534, 0.04523762117838806, 0.39867523, 0.39920786, 0.77604514, -0.700944, -0.79042625, -0.22849922), ('a,gear,stand', 0.022146662344973197, -0.032867323805555716, -0.07143672518383183, 0.07314944656109809, 0.055702438543511444, 0.05017749590092538, 0.81039995, 0.35446674, 1.1475562, -2.0383835, -0.55287224, -1.2319783), ('a,gear,null', 0.022570096625655994, -0.036608760294460654, -0.07169493794752187, 0.01779986380982108, 0.011561341072093041, 0.008039675979049308, 0.0561927, -0.018109497, -0.055127434, -0.0314253, -0.08042747, -0.09347696)]


In [159]:
def process_further2(filename):
    rdd = read_in(filename)
    mean = calc_mean(rdd).map(lambda x : (x[0], list(x[1])))
    std = calc_std(rdd).map(lambda x : (x[0], list(x[1])))
    max = find_max(rdd).map(lambda x : (x[0], list(x[1])))
    min = find_min(rdd).map(lambda x : (x[0], list(x[1])))

    #out = mean + std + max + min
    out = mean.union(std).union(max).union(min).groupByKey().map(lambda x : (x[0], list(x[1])))
    #out = out.groupByKey().map(lambda x : (x[0], list(x[1])))
    #.map(lambda x : (x[0], x[1])).join(min)
    #out = out.map(lambda x : (x[0], x[1][0][0][0][0], x[1][0][0][0][1], x[1][0][0][0][2], x[1][0][0][1][0], x[1][0][0][1][1], x[1][0][0][1][2], x[1][0][1][0], x[1][0][1][1], x[1][0][1][2], x[1][1][0], x[1][1][1], x[1][1][2]))
    #out = out.map(lambda x : np.split(x[1], ","))
    #out = out.map(lambda x : (x[0], x[1][0][0], x[1][1]))
    return out

In [160]:
out = process_further2('Watch_gyroscope.csv')
print(out.collect())

[('a,gear,sit', [[0.02270561507082863, -0.04142249651518432, -0.06997320145904558], [0.057033688206817146, 0.056269236352008534, 0.04523762117838806], [0.39867523, 0.39920786, 0.77604514], [-0.700944, -0.79042625, -0.22849922]]), ('a,gear,stand', [[0.022146662344973197, -0.032867323805555716, -0.07143672518383183], [0.07314944656109809, 0.055702438543511444, 0.05017749590092538], [0.81039995, 0.35446674, 1.1475562], [-2.0383835, -0.55287224, -1.2319783]]), ('a,gear,null', [[0.022570096625655994, -0.036608760294460654, -0.07169493794752187], [0.01779986380982108, 0.011561341072093041, 0.008039675979049308], [0.0561927, -0.018109497, -0.055127434], [-0.0314253, -0.08042747, -0.09347696]])]
